In [ ]:

import numpy as np
from scipy.fft import rfft, rfftfreq
import matplotlib.pyplot as plt

In [ ]:
def lorentzian(f, f0, hwhm, snr):
    return snr / (1 + ((f - f0) / hwhm)**2)



def gen_spectrum(species='anolis', noise_floor=[], noise_amp=0.01, n_peaks=10):
    # Get frequency axis
    f = rfftfreq(32768, 1/44100)
    
    # Get num bins in the spectrum
    n_bins = len(noise_floor)
    if n_bins != 8192:
        raise ValueError(f"Expected noise floor to be 8192 bins, but it's {n_bins} bins!")
    
    # Initialize matrix to store all the different peaks to be added to the noisefloor
    spec_components = np.empty((n_peaks + 1, n_bins))
    
    # Add noise floor
    spec_components[-1, :] = noise_floor
    
    # Set generation parameters
    
    # f0: we'll draw half from a chi square
    f0_chi_dof = 10
    f0_chi_og_pivot = 20 # This is the value of the original distribution that we want to "grab"
    f0_chi_new_pivot = 8000 # We'll rescale the distribution so that that value becomes this value
    # and the other half from a uniform distribution
    f0_min = 50
    f0_max = 6000
    
    # hwhm: we'll draw from uniform distribution
    if species == 'Anolis':
        hwhm_min = 25
        hwhm_max = 125
    elif species == 'Human':
        hwhm_min = 3
        hwhm_max = 20

    # SNR: we'll draw from a uniform distribution
    snr_min = 0
    snr_max = 25
    
    # Create a Generator instance
    rng = np.random.default_rng()  # Default random generator instance
    
    # Generate and add peaks
    for i in range(n_peaks):
        # f0
        # Draw half of the positions from a chi square
        if i < n_peaks / 2:
            f0 = rng.chisquare(df=f0_chi_dof)*f0_chi_new_pivot/f0_chi_og_pivot
            # make sure it's below our spectral max, resample if not
            while f0 > f[-1]:
                f0 = rng.chisquare(df=f0_chi_dof)*f0_chi_new_pivot/f0_chi_og_pivot
        # And the other half from a uniform distribution
        else:
            f0 = rng.uniform(f0_min, f0_max)
        
        # hwhm
        hwhm = rng.uniform(hwhm_min, hwhm_max)
        
        # SNR
        snr = rng.uniform(snr_min, snr_max)
        
        # Add peak
        spec_components[i, :] = lorentzian(f, f0, hwhm, snr)
        

        
        

In [ ]:
# Parameters for the chi-square distribution
f0_chi_dof = 10  # k (degrees of freedom)
num_samples = 10000     # Number of samples to generate

# Generate samples from a chi-square distribution
f0_chi_og_pivot = 20
f0_chi_new_pivot = 6000
chi_square_samples = np.random.chisquare(df=f0_chi_dof, size=num_samples)*f0_chi_new_pivot/f0_chi_og_pivot

# Plot the histogram of the samples
plt.hist(chi_square_samples, bins=50, density=True, alpha=0.7, color='blue', edgecolor='black')
plt.title(f'Chi-Square Distribution (k={f0_chi_dof})')
plt.xlabel('Value')
plt.ylabel('Density')
plt.grid(True)
plt.show()

In [ ]:
f = rfftfreq(32768, 1/44100)
f0 = 5000
snr = 1
hwhm = 1000

plt.plot(f, lorentzian(f, f0, hwhm, snr))
plt.show()

In [48]:
import statsmodels.api as sm
import pandas as pd
import os

# Get noise floor files

# Get frequency axis
f = rfftfreq(32768, 1/44100)[0:8192]

# First navigate to our directory
directory_path = os.path.join("Data", "processed_df.parquet")
# Load the dataframe
df = pd.read_parquet(directory_path)
# === deal w/ loess
lowess = sm.nonparametric.lowess
sigma= 0.08  # local-ness factor {0.1-0.2}

i = 0

for indx, row in df.iterrows():
    i+=1
    spectrum = row['spectrum']
    fit= lowess(spectrum,f/1000,frac=sigma, return_sorted=False)
    plt.plot(f, spectrum)
    plt.plot(f, fit,'g--',label='loess fit')
    plt.title(row['filepath'])
    plt.show()
    
    if i > 10:
        break
    




KeyboardInterrupt: 